### Doing a test on different was to remove the seasonal variability

In [3]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr
import cmocean.cm as cmocean
from collections import OrderedDict
import cartopy.crs as ccrs
import matplotlib.path as mpath
import cftime
import glob
import dask
import os
import datetime
import matplotlib.colors as col
import gsw
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client
# from cosima_cookbook import distributed as ccd


In [5]:
from dask.distributed import Client

print('starting distributed client...')
client = Client()
display(client)

starting distributed client...


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/42035/status,
Dashboard: /proxy/42035/status,Workers: 7
Total threads: 28,Total memory: 128.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35281,Workers: 7
Dashboard: /proxy/42035/status,Total threads: 28
Started: Just now,Total memory: 128.00 GiB
Comm: tcp://127.0.0.1:46255,Total threads: 4
Dashboard: /proxy/37567/status,Memory: 18.29 GiB
Nanny: tcp://127.0.0.1:40313,


In [6]:
master_session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')
iaf_cycle3 = '01deg_jra55v140_iaf_cycle3'

first_year = '1958'
last_year = '2018'

start_time=first_year+'-01-01'
end_time=last_year+'-12-31'

time_period = str(int(start_time[:4]))+'-'+str(int(end_time[:4]))

#### Creating variables for time series
#### TEMPERATURE BOX 1

In [7]:
# load temperature data at 400m:

depth_slice = slice(400, 6000)
# cut the data BOX 1
lon_slice  = slice(-252.7, -251.7)
lat_slice  = slice(-65.35, -65.1)

temp_400m = cc.querying.getvar(iaf_cycle3,'temp' , master_session,frequency='1 monthly',start_time=start_time, end_time=end_time) - 273.15
temp_400m = temp_400m.sel(yt_ocean=lat_slice).sel(xt_ocean=lon_slice).sel(st_ocean=depth_slice)

In [8]:
# depth average:
# there is no dzu saved
# calculate grid thickness, adjusted for partial cells:
zlevel1 = 400
zlevel2 = 6000

# import edges of st_ocean and add lat/lon dimensions:
st_edges_ocean = cc.querying.getvar(iaf_cycle3,'st_edges_ocean',master_session,
                 start_time=start_time, end_time=end_time,n=1)
st_edges_array = st_edges_ocean.expand_dims({'yt_ocean':temp_400m.yt_ocean,'xt_ocean':temp_400m.xt_ocean},axis=[1,2])
# import bathymetry:
ht = cc.querying.getvar(iaf_cycle3,'ht',master_session,n=1)
ht = ht.sel(yt_ocean=lat_slice).sel(xt_ocean=lon_slice)
# adjust edges at bottom for partial thickness:
st_edges_with_partial = st_edges_array.where(st_edges_array<ht,other=ht)
thickness = st_edges_with_partial.diff(dim='st_edges_ocean')

# change coordinate of thickness to st_ocean (needed for multipling with other variables):
st_ocean = cc.querying.getvar(iaf_cycle3,'st_ocean',master_session,n=1)
thickness['st_edges_ocean'] = st_ocean.values
thickness = thickness.rename(({'st_edges_ocean':'st_ocean'}))

# select correct depths:
thickness = thickness.sel(st_ocean=slice(zlevel1,zlevel2))

# multiply by thickness to avg
temp_400m = (temp_400m*thickness).sum('st_ocean')/thickness.sum('st_ocean')

In [16]:
##calculating the month climatology
temp_clim = temp_400m.groupby('time.month').mean('time')

In [18]:
#load the variable
temp_400m_za = temp_400m.load()
temp_clim = temp_clim.load()
##test
#np.amin(temp_400m_za)

In [19]:
#removing the seasonality 
clim=temp_400m-temp_clim

#### Detrend time series and apply rolling mean 

In [20]:
# Note we need to first detrend the data:
clim_zad = detrend_dim(clim,'time')
clim_zadr = clim_zad.rolling(time=12, center=True).mean()
clim_zar = clim_za.rolling(time=12, center=True).mean()

NameError: name 'detrend_dim' is not defined

In [15]:
fontsize=18
fig=plt.figure(figsize=(21, 22))
fig.subplots_adjust(top=0.88)

ax1 = plt.subplot(4, 1, 2)
temp_400m_zar.plot(color='darkorange')
#temp_high_mask.plot(color='red')
#temp_low_mask.plot(color='purple')
temp_400m_zar[bt_id].plot(marker='.',color='blue',linestyle='none')
temp_400m_zar[top_id].plot(marker='.',color='red',linestyle='none')

ax1.set_ylabel('Intrusion Box - T > 400m \n ($^\circ$C)', fontsize=fontsize,color='darkorange')
ax1.tick_params(axis='y', labelcolor='darkorange',labelsize=fontsize)
ax1.set_xlabel(' ', fontsize = fontsize)
ax1.tick_params(axis='x',labelsize=fontsize)
#plt.title('Monthly T (Box1) vs Zonal Wind (small grid; -65,-64.5S/-258,-252E) / Rolling mean',fontsize=fontsize)
plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5)
plt.ylim([-1.6,-0.2])
#plt.text(100,0.4,"cor = -0.73, lag = 4 months")
#ax1.xaxis.set_minor_locator(MultipleLocator(1))
plt.plot(np.nan,np.nan,color='k',marker=".",linestyle='none',label='cor = - 0.7')
plt.legend(loc="lower left",fontsize=fontsize)

ax2 = ax1.twinx()
#isocon_off_dr.plot(color='C0')
isocon2_off_r.plot(color='C0',linestyle='--')
#isocon_off3_dr.plot(color='C0',linestyle=':')
ax2.set_ylabel('Isopycnal Depth (m)', fontsize=fontsize, color='C0')
ax2.tick_params(axis='x',labelsize=fontsize)
ax2.tick_params(axis='y', labelcolor='C0',labelsize=fontsize)
#ax2.invert_yaxis()
plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5)
#plt.text(700, -0.4, 'cor=0.016', fontsize = 12)
#building legend
#plt.plot(np.nan,np.nan,color='C0',linestyle=':',label='27.80')
plt.plot(np.nan,np.nan,color='C0',linestyle='--',label='27.82 kgm$^{-3}$')
#plt.plot(np.nan,np.nan,color='C0',label='27.83')
plt.legend(loc="upper left",fontsize=fontsize)
plt.ylim([200,900])

#figdir = '/g/data/jk72/ns3783'
#figdir = '/home/581/ns3783/Scripts/Figures/New_codes_CFD/Figures'
#filestr = figdir+'/Fig3_timeseries.png'
#plt.savefig(filestr,dpi=600,bbox_inches='tight')

,Array,Chunk
Bytes,3.35 MiB,400 B
Shape,"(732, 5, 10, 12)","(1, 5, 10, 1)"
Count,3599829 Tasks,8784 Chunks
Type,float64,numpy.ndarray
